# Chinese Word Segmentation (ckiptagger)



In [1]:
DEMO_DATA_ROOT="../../../RepositoryData/data"

The current state-of-art Chinese segmenter for Taiwan Mandarin available is probably the [CKIP tagger](https://github.com/ckiplab/ckiptagger), created by the [Chinese Knowledge and Information Processing (CKIP)](https://ckip.iis.sinica.edu.tw/) group at the Academia Sinica.

The `ckiptagger` is released as a python module.

The normal CPU version is very slow. Not sure if it is the case for GPU version.

## Installation

```
!pip install ckiptagger
```

```{note}
Remember to download the model files. Very big.
```

## Download the Model Files

All NLP applications have their models behind their fancy performances. To use the tagger provided in `ckiptagger`, we need to download their pre-trained model files. 

Please go to the [github of CKIP tagger](https://github.com/ckiplab/ckiptagger) to download the model files, which is provided as a zipped file. (The file is very big. It takes a while.)

After you download the zipped file, unzip it under your working directory to the `data/` directory.

## Segmenting Texts

The initialized word segmenter object, `ws()`, can tokenize any input **character vectors** into a list of **word vectors** of the same size.

In [2]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

```{note}
Please use own model path. The model files are very big. They are probably not on the Drive.
```

In [3]:
# Set Parameter Path
MODEL_PATH = '../../../../../Dropbox/Corpus/CKIP_WordSeg/data/'
ws = WS(MODEL_PATH)
pos = POS(MODEL_PATH)
ner = NER(MODEL_PATH)

In [7]:
## Raw text corpus 
sentence_list = ['傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。',
              '美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會，預料她將會很順利通過參議院支持，成為該國有史以來第一位的華裔女性內閣成員。',
              '土地公有政策?？還是土地婆有政策。',
              '… 你確定嗎… 不要再騙了……他來亂的啦',
              '最多容納59,000個人,或5.9萬人,再多就不行了.這是環評的結論.',
              '科長說:1,坪數對人數為1:3。2,可以再增加。']
    ## other parameters
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced

word_list = ws(sentence_list)
pos_list = pos(word_list)
entity_list = ner(word_list, pos_list)
    

In [8]:
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
    
for i, sentence in enumerate(sentence_list):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_list[i],  pos_list[i])
    for entity in sorted(entity_list[i]):
        print(entity)



'傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
傅達仁(Nb)　今(Nd)　將(D)　執行(VC)　安樂死(Na)　，(COMMACATEGORY)　卻(D)　突然(D)　爆出(VJ)　自己(Nh)　20(Neu)　年(Nf)　前(Ng)　遭(P)　緯來(Nb)　體育台(Na)　封殺(VC)　，(COMMACATEGORY)　他(Nh)　不(D)　懂(VK)　自己(Nh)　哪裡(Ncd)　得罪到(VJ)　電視台(Nc)　。(PERIODCATEGORY)　
(0, 3, 'PERSON', '傅達仁')
(18, 22, 'DATE', '20年前')
(23, 28, 'ORG', '緯來體育台')

'美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會，預料她將會很順利通過參議院支持，成為該國有史以來第一位的華裔女性內閣成員。'
美國(Nc)　參議院(Nc)　針對(P)　今天(Nd)　總統(Na)　布什(Nb)　所(D)　提名(VC)　的(DE)　勞工部長(Na)　趙小蘭(Nb)　展開(VC)　認可(VC)　聽證會(Na)　，(COMMACATEGORY)　預料(VE)　她(Nh)　將(D)　會(D)　很(Dfa)　順利(VH)　通過(VC)　參議院(Nc)　支持(VC)　，(COMMACATEGORY)　成為(VG)　該(Nes)　國(Nc)　有史以來(D)　第一(Neu)　位(Nf)　的(DE)　華裔(Na)　女性(Na)　內閣(Na)　成員(Na)　。(PERIODCATEGORY)　
(0, 2, 'GPE', '美國')
(2, 5, 'ORG', '參議院')
(7, 9, 'DATE', '今天')
(11, 13, 'PERSON', '布什')
(17, 21, 'ORG', '勞工部長')
(21, 24, 'PERSON', '趙小蘭')
(42, 45, 'ORG', '參議院')
(56, 58, 'ORDINAL', '第一')
(60, 62, 'NORP', '華裔')

'土地公有政策?？還是土地婆有政策。'
土地公有(VH)　政策(Na)　?(QUESTIONCATEGORY)　？(QUESTIONCATEGORY)　還是(Caa)　土地(Na)　婆(Na)　有(V

## Define Own Dictionary

The performance of Chinese word segmenter depends highly on the dictionary. Texts in different disciplines may have very domain-specific vocabulary. To prioritize a set of words in a dictionary, we can further ensure the accuracy of the word segmentation.

To create a dictionary for `ckiptagger`:

In [9]:

word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "": 1,
    "來亂的": "啦",
    "緯來體育台": 1,
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

word_list_2 = ws(sentence_list,
                recommend_dictionary=dictionary)
print(word_list)
print(word_list_2)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0}), (5, {'緯來體育台': 1.0})]
[['傅達仁', '今', '將', '執行', '安樂死', '，', '卻', '突然', '爆出', '自己', '20', '年', '前', '遭', '緯來', '體育台', '封殺', '，', '他', '不', '懂', '自己', '哪裡', '得罪到', '電視台', '。'], ['美國', '參議院', '針對', '今天', '總統', '布什', '所', '提名', '的', '勞工部長', '趙小蘭', '展開', '認可', '聽證會', '，', '預料', '她', '將', '會', '很', '順利', '通過', '參議院', '支持', '，', '成為', '該', '國', '有史以來', '第一', '位', '的', '華裔', '女性', '內閣', '成員', '。'], ['土地公有', '政策', '?', '？', '還是', '土地', '婆', '有', '政策', '。'], ['…', ' ', '你', '確定', '嗎', '…', ' ', '不要', '再', '騙', '了', '…', '…', '他', '來', '亂', '的', '啦'], ['最多', '容納', '59,000', '個', '人', ',', '或', '5.9萬', '人', ',', '再', '多', '就', '不行', '了', '.', '這', '是', '環評', '的', '結論', '.'], ['科長', '說', ':1,', '坪數', '對', '人數', '為', '1:3', '。', '2', ',', '可以', '再', '增加', '。']]
[['傅達仁', '今', '將', '執行', '安樂死', '，', '卻', '突然', '爆出', '自己', '20', '年', '前', '遭', '緯來體育台', '封殺', '，', '他', '不', '懂', '自己', '哪裡', '得罪到', '電視台', '。'], ['美國', '參議院', '針對', '今天', '總統', '布什', '所', '提名

## Convert ckiptagger output into a Data Frame?